# Libs

In [1]:
import os
# while os.getcwd().split('\\')[-1] != 'master-llm-rag-vnlaw':
while os.getcwd().split('\\')[-1] != 'master-llm-rag-vnlaw':
    os.chdir('..')
    print(os.getcwd())

d:\DANG NHI\repos\master-llm-rag-vnlaw


In [2]:
import os
import regex as re
import random
import json
import pandas as pd
import numpy as np

from itertools import combinations
from itertools import product

In [3]:
base_model_id = "FacebookAI/xlm-roberta-base"

# Load tokenizer

In [4]:
import torch
from transformers import AutoTokenizer, AutoModel

In [5]:
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
model = AutoModel.from_pretrained(base_model_id)

# Chunking

In [7]:
# data_path = "/kaggle/input/vn-law-test/data"
data_path = "data/"

MAX_LEN = 512
OVERLAP = 100

In [8]:
files = os.listdir(data_path)
data_stored = []

In [9]:
# khi split chunking không quá quan tâm đến độ dài
# mỗi chunk phải là một đoạn hoặc một câu hoàn chỉnh
# nếu là bảng thì phải tách riêng

def preprocess_phrase_chunking(text):
    s = text
    s = re.sub('CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM', '', s)
    s = re.sub('Độc lập - Tự do - Hạnh phúc', '', s)
    s = re.sub('[- ]+', ' ', s)
    s = s.strip()
    return s

def count_tokens(text):
    return len(tokenizer.encode(text))

def token_chunking(paragraphs, max_len=512, overlap=100):

    pass

### test 1 chunk

In [41]:
file = files[1]

with open(f"{data_path}/{file}", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

In [42]:
data = raw_data[random.randint(0, len(raw_data))]

In [43]:
content = data['content']
for i in range(5):
    content[i] = preprocess_phrase_chunking(content[i])

In [44]:
counts = [count_tokens(c) for c in content]
print(counts)

[38, 9, 41, 23, 16, 18, 33, 97, 31, 52, 50, 94, 14, 10, 41, 101, 106, 186, 6, 69, 4, 12, 13, 98, 204, 114, 11, 94, 36, 119, 76, 56, 10, 140, 96, 96, 31, 8, 17, 22, 15, 9, 73, 75, 40, 48, 33, 20, 23, 56, 16, 32, 71, 32, 39, 57, 28, 68, 34, 61, 52, 33, 76, 4, 22, 15, 104, 15, 161, 64, 64, 96, 62, 69, 13, 37, 42, 41, 36, 32, 74, 27, 37, 28, 26, 39, 29, 86, 4, 45, 19, 181, 195, 168, 49, 93, 55, 104, 68, 82, 76, 37, 277, 78, 67, 107, 111, 26, 131, 121, 120, 94, 15, 89, 65, 141, 4, 25, 25, 127, 88, 120, 14, 171, 43, 44, 52, 64, 40, 36, 37, 20, 30, 51, 82, 64, 87, 16, 47, 58, 50, 21, 50, 68, 4, 19, 14, 10, 32, 50, 111, 19, 18, 64, 33, 102, 143, 129, 104, 47, 171, 22, 67, 116, 11, 53, 65, 66, 83, 14, 38, 14, 9, 144, 69, 10, 168, 93, 78, 195, 92, 90, 121, 106, 15, 16, 28, 38, 55, 62, 41, 58, 127, 30, 150, 77, 8, 17, 35, 75, 88, 51, 10, 123, 92, 90, 252, 92, 83, 68, 12, 15, 94, 32, 83, 16, 121, 10, 69, 143, 45, 96, 63, 142, 58, 25, 101, 18, 62, 50, 62, 77, 83, 88, 82, 137, 122, 16, 44, 27, 84, 4

In [52]:
def chunking_long_text(input_text, remain_text='', 
                       max_len=512, overlap=100):
    chunks = []
    rows = input_text.split("\n")
    current_text = remain_text

    counts = [count_tokens(r) for r in rows]
    counts[0] = counts[0] + count_tokens(remain_text)

    current_chunk_idx = []
    for i in range(len(counts)):
        if sum([counts[i] for i in current_chunk_idx]) + counts[i] < MAX_LEN - 10:
            current_chunk_idx.append(i)
        else:
            current_text += "\n".join([rows[j] for j in current_chunk_idx])
            current_text = current_text.strip() + "\n (bảng còn tiếp)"
            chunks.append(current_text)

            j = 1
            while sum([counts[i] for i in current_chunk_idx[-j:]]) < OVERLAP - 10:
                j += 1

            current_text = " \n (tiếp tục bảng) \n" + "\n".join([rows[j] for j in current_chunk_idx[-j:]]) + "\n"
            current_chunk_idx = current_chunk_idx[-j:] + [i]

    if len(current_chunk_idx) > 0:
        current_text += "\n".join([rows[j] for j in current_chunk_idx])
        chunks.append(current_text)
    return chunks

# results = chunking_long_text(content[47], max_len=512, overlap=100)

def last_words(text, n=10):
    lst = text.split(" ")[-n:]
    return " ".join(lst)

def token_chunking(content, max_len=512, overlap=100):
    chunks = []

    curr_content = ""
    curr_group_idx = []

    for i in range(len(content)):
        if sum([counts[i] for i in curr_group_idx]) + counts[i] < MAX_LEN:
            curr_group_idx.append(i)
        else:
            curr_content = curr_content + "\n" + "\n".join([content[j] for j in curr_group_idx])
            curr_content = curr_content.strip()
            chunks.append(curr_content)

            # split if text i too long
            if counts[i] > MAX_LEN:
                curr_content = last_words(curr_content, n=OVERLAP//2)
                small_split = chunking_long_text(content[i], curr_content , MAX_LEN, OVERLAP)
                chunks += small_split
                curr_content = last_words(small_split[-1], n= 10)
                curr_group_idx = []
            else:
                j = 1
                while (sum([counts[i] for i in curr_group_idx[-j:]]) < OVERLAP) :
                    j += 1
                curr_group_idx = curr_group_idx[-j:] + [i]
                curr_content = ""
    if len(curr_group_idx) > 0:
        curr_content = curr_content + "\n" + "\n".join([content[j] for j in curr_group_idx])
        curr_content = curr_content.strip()
        chunks.append(curr_content)
    return chunks

In [53]:
results = token_chunking(content, max_len=512, overlap=100)

In [57]:
print(len(results))
print(results[-1])

83
2. Trong thời hạn 15 ngày kể từ ngày văn bản hợp tác quốc tế được ký kết,...(2)... báo cáo Bộ trưởng Bộ Văn hóa, Thể thao và Du lịch về việc ký, đồng thời gửi Cục Hợp tác quốc tế bản sao văn bản hợp tác quốc tế.
3. ...(2)... chủ trì xây dựng, triển khai kế hoạch thực hiện văn bản hợp tác quốc tế và định kỳ báo cáo Bộ trưởng Bộ Văn hóa, Thể thao và Du lịch kết quả thực hiện văn bản hợp tác quốc tế.
Điều 3. Chánh Văn phòng Bộ, Vụ trưởng Vụ Kế hoạch, Tài chính, Cục trưởng Cục Hợp tác quốc tế,...(2)... và Thủ trưởng các cơ quan liên quan chịu trách nhiệm thi hành Quyết định này./.
Bắt đầu bảng: 
Nơi nhận:- Như Điều 3;- Bộ trưởng;- Lưu: VT, ....(2),...... | BỘ TRƯỞNG
 Kết thúc bảng.
Chú thích :
(1). Ghi số Quyết định ban hành Quy chế Quản lý hoạt động đối ngoại của Bộ Văn hóa, Thể thao và Du lịch;
(2). Ghi tên đơn vị đề xuất ký kết văn bản hợp tác quốc tế;
(3). Ghi tên văn bản hợp tác quốc tế quốc tế;
(4).

In [55]:
content

['BỘ VĂN HOÁ, THỂ THAO VÀ DU LỊCH Số: 570/QĐ BVHTTDL Hà Nội, ngày 11 tháng 03 năm 2024',
 'QUYẾT ĐỊNH',
 'BAN HÀNH QUY CHẾ QUẢN LÝ HOẠT ĐỘNG ĐỐI NGOẠI CỦA BỘ VĂN HÓA, THỂ THAO VÀ DU LỊCH',
 'BỘ TRƯỞNG BỘ VĂN HOÁ, THỂ THAO VÀ DU LỊCH',
 'Căn cứ Luật Điều ước quốc tế ngày 09 tháng 4 năm 2016;',
 'Căn cứ Luật Thỏa thuận quốc tế ngày 13 tháng 11 năm 2020;',
 'Căn cứ Nghị định 72/2015/NĐ-CP ngày 07 tháng 9 năm 2015 của Chính phủ quy định về quản lý hoạt động thông tin đối ngoại;',
 'Căn cứ Nghị định số 64/2021/NĐ-CP ngày 30 ngày 6 năm 2021 của Chính phủ quy định việc ký kết và thực hiện thỏa thuận quốc tế nhân danh Tổng cục, Cục thuộc bộ, cơ quan ngang bộ; cơ quan chuyên môn thuộc Ủy ban nhân dân cấp tỉnh; Ủy ban nhân dân cấp huyện; Ủy ban nhân dân cấp xã ở khu vực biên giới; cơ quan cấp tỉnh của tổ chức;',
 'Căn cứ Nghị định số 18/2022/NĐ-CP ngày 18 tháng 02 năm 2022 của Chính phủ quy định về nghi lễ đối ngoại;',
 'Căn cứ Nghị định số 01/2023/NĐ-CP ngày 1

# Embeddings

In [ ]:
def embedding_preprocess(text):
    s = text
    s = re.sub(r"\|", " ", s)
    # s = re.sub(r"[\d\.-]+", "number", s)
    s = re.sub(r"\d[\d,.A-Za-x]{2,}", '', s)
    # s = re.sub(r'\b\d+(?:[,.]\d+)?%?\b|\b\d+[A-Za-z]\d*(?:\.\d+)*\b', 'number', s)
    s = s.strip()
    return s

def embedding(text):
    input_ids = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
    with torch.no_grad():
        output = model(**input_ids)
    return output

embed = embedding_preprocess(table_text)
embed = embedding(embed)

In [83]:
(count_tokens(table_text), count_tokens(embed))

(62119, 32121)

In [21]:
tokens = tokenizer.encode_plus(
    texts[7], 
    truncation=True, 
    padding='max_length',
    return_tensors='pt'
    )
tokens['input_ids'].shape


NameError: name 'texts' is not defined

In [ ]:
# def tokenizer_chunking(content, tokenizer, max_len=512, overlap=50):
#     tokens = tokenizer.encode(content, add_special_tokens=False)
#     chunks = []
#     start = 0

#     while start < len(tokens):
#         end = min(start + max_len, len(tokens))
#         chunk = tokens[start:end]
#         chunks.append(chunk)
#         start += max_len - overlap

#     return chunks

# # Example usage
# tokenized_chunks = tokenizer_chunking(content, tokenizer, max_len=MAX_LEN, overlap=OVERLAP)
# print(f"Number of chunks: {len(tokenized_chunks)}")

512